<a href="https://colab.research.google.com/github/SotaYoshida/Lecture_DataScience/blob/main/notebooks/Python_chapter_ArtificialNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習における最適化

この章では、最適化手法、中でもニューラルネットワークの訓練のための連続最適化手法を中心に解説する。

理がない限りニューラルネットワークの構造としてはシンプルな多層パーセプトロン(MLP)を対象とし、損失関数としては回帰問題に対する平均二乗誤差、分類問題に対する交差エントロピー誤差を想定しているが、なるべく一般的な内容を扱うようにする。

また、現代においても新たな最適化手法が提案され続けているため、ここでは代表的な手法を紹介するにとどめる。

## 勾配降下法


勾配降下法(Gradient Descent, GD)は、関数の最小値を見つけるための最も基本的な最適化手法である。

関数$f(\boldsymbol{\theta})$の勾配$\nabla f(\boldsymbol{\theta})$は、関数が最も急激に増加する方向を示すベクトルであり、
勾配降下法ではこの勾配の反対方向にパラメータ$\boldsymbol{\theta}$を更新することで、関数の値を減少させる、というのが基本的な考え方である。

最も単純なものが最急降下法であり、パラメータ$\boldsymbol{\theta}$の更新は以下のように行われる。

$$
W_i := W_i - \eta \frac{\partial L}{\partial W_i}
$$

$L$は目的関数で、$\eta$は学習率(パラメータ更新のスケールを決めるパラメータ)とした。

ANN(MLP)の最適化においてはしばし、訓練データを小さなサブ集合(ミニバッチ)に分割し、各ミニバッチに対して勾配降下法を適用することが行われる。
加えて、ミニバッチの中からランダムにサンプルを選ぶことも多く、これを確率的勾配降下法(Stochastic Gradient Descent, SGD)と呼ぶ。
これにより、計算コストの削減と汎化性能の向上が期待できる。

SGDなどの勾配法はシンプルで実装も容易であるが、中々収束しない、局所最適解に陥りやすい、などの問題が生じることもある。
そうした経緯から提案されたのが、勾配だけでなく、過去の更新情報を利用するモーメンタム法や、パラメータごとに異なる学習率を適応的に調整するAdaGrad, RMSProp, Adamなどの手法である。
年代に沿ってこれらの手法を紹介するのも楽しいが、続く節では、現代においても広く用いられているAdamを中心に説明する。




## Adam

Adamは、勾配降下法の様にその都度の勾配の情報だけを使うのではなく、以前の勾配の情報も有効活用しながら学習率も調整する手法である。

2014年にDiederik P. KingmaとJimmy Baによって提案されて以来、深層学習の分野で現代に至るまで広く用いられている。→[arXiv:1412.6980](https://arxiv.org/abs/1412.6980)





### AdamW

AdamWは、Adamの変種であり、L2正則化を直接パラメータ更新に組み込むことで、過学習を防止する手法である。
名前のWはWeight decay(重み減衰)に由来する。このweight decayは過学習を防止するための正則化手法の一つであり、パラメータの大きさを抑制するために用いられる。
損失関数にパラメータの二乗和を加えることで実現される。

これは機械学習分野ではL2正則化と呼ばれるが、SGDの場合と異なり、Adamに対して単純にパラメータ更新の式にL2正則化項を加えてしまうと、重み減衰の効果が正しく働かないことが指摘された。
そこで提案されたのがAdamWである。


## 二階微分を用いる方法

勾配のみを用いる手法は、関数の形状に関する情報が限られているため、収束が遅い、局所最適解に陥りやすい、などの問題がある。これに対処するために、ヘッセ行列を用いた二階微分の情報を活用する方法がある。
二階微分の情報を用いることで、関数の曲率に関する情報を得ることができ、より効率的な最適化が可能になる。
一方で、ヘッセ行列の計算は計算コストが高く、特に高次元のパラメータ空間では実用的でないことが多い。

代表的な手法として、ニュートン法や準ニュートン法(L-BFGS法, etc.)などがある。
ニュートン法では、ヘッセ行列の逆行列を用いてパラメータを更新するのに対して、
準ニュートン法では、ヘッセ行列の近似を用いることで計算コストを削減する違いがある。


## 残差接続

最適化手法とは異なるが、ニューラルネットワークの構造として残差接続(Residual Connection)が提案されている。





## $\clubsuit$ EMアルゴリズム

Expectation maximization (EM)アルゴリズムは、観測データに対して潜在変数が存在する場合に、パラメータの最尤推定を行うための反復的な手法である。
とくに、混合ガウスモデルや隠れマルコフモデルなどの確率モデルで広く用いられている他、
最近では、変分オートエンコーダ(Variational Autoencoder, VAE)などの深層生成モデルにも応用されている。

以下では、複数のガウス分布の線形結合でデータをモデル化する**ガウス混合モデル**を例に、EMアルゴリズムの基本的な考え方を説明する。

初めに、各ガウス分布のパラメータを $\boldsymbol{\mu}_k$ (平均)、$\boldsymbol{\Sigma}_k$ (共分散行列)、および混合係数 $\pi_k$ (各ガウス分布の寄与度) とする。

* **Eステップ**: 現在のパラメータを用いて、各データ点$n$がどのガウス分布$k$に属するかの確率を計算する。
    
    $$
    r_{nk} = \frac{\pi_k \mathcal{N}(\boldsymbol{x}_n | \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k)}{
        \sum_j \pi_j  \mathcal{N}(\boldsymbol{x}_n | \boldsymbol{\mu}_j, \boldsymbol{\Sigma}_j)
    }
    $$

* **Mステップ**: Eステップで得られた確率$r$を用いて、ガウス分布のパラメータ $\boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k, \pi_k$ を更新する。

    $$
    \boldsymbol{\mu}_k = \frac{1}{N_k} \sum_n r_{nk} \boldsymbol{x}_n, \quad
    \boldsymbol{\Sigma}_k = \frac{1}{N_k} \sum_n r_{nk} (\boldsymbol{x}_n - \boldsymbol{\mu}_k)(\boldsymbol{x}_n - \boldsymbol{\mu}_k)^\top, \quad
    \pi_k = \frac{N_k}{N}
    $$

この逐次更新によって対数尤度の最大化を図り、収束するまでEステップとMステップを繰り返す。
対数尤度は以下のように表される。

$$
\mathcal{L}(\boldsymbol{\theta}) = \sum_n \log \left( \sum_k \pi_k \mathcal{N}(\boldsymbol{x}_n | \boldsymbol{\mu}_k, \boldsymbol{\Sigma}_k) \right)
$$